# Clasificación de imágenes con red profunda

Veamos ahora como se desempeña una red profunda con el ejemplo de clasificación sobre CIFAR-10

Para empezar importamos las librerías iniciales

In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Importamos la base de imágenes, tal cual hicimos en las clases anteriores

In [ ]:
from data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):

    # Cargados de datos
    cifar10_dir = 'cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # Sub-muestreo
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    return X_train, y_train, X_val, y_val, X_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = get_CIFAR10_data(4900, 100, 100)

print('Tamaño de datos de entrenamiento: ', x_train.shape)
print('Tamaño de etiquetas de entrenamiento: ', y_train.shape)
print('Tamaño de datos de validación: ', x_val.shape)
print('Tamaño de etiquetas de validación: ', y_val.shape)
print('Tamaño de datos de test: ', x_test.shape)
print('Tamaño de etiquetas de test: ', y_test.shape)

Visualizamos algunos ejemplos

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

## Creamos la red, utilizando Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Elegimos optimizador
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Creamos la red efectivamente
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Normalización
mean_image = np.mean(x_train, axis=0)

x_train = (x_train - mean_image).astype('float32') / 255
x_test = (x_test - mean_image).astype('float32') / 255
x_val = (x_val - mean_image).astype('float32') / 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Entrenamiento

In [ ]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = False
num_predictions = 20

print('Not using data augmentation.')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

### Guardamos el modelo y sus pesos

In [ ]:
model_path = "keras_cifar10_trained_model.h5"
model.save(model_path)
print('Saved trained model at %s ' % model_path)

### Evaluamos el resultado

In [ ]:
from keras.models import load_model

model = load_model('keras_cifar10_trained_model.h5')

scores = model.evaluate(x_val, y_val, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Fin!